<a href="https://colab.research.google.com/github/ipeirotis/dealing_with_data/blob/master/11-Flask/I-All_Elements_Together.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# All pieces together

This is just a notebook that puts together all the pieces of the code that we have been writing during this tutorial.

In [ ]:
!pip install -U PyMySQL sqlalchemy flask pyngrok geopandas pygeos

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import threading

from flask import Flask, render_template, jsonify, request
from pyngrok import ngrok

from sqlalchemy import create_engine
import pandas as pd
import base64
from io import BytesIO
import matplotlib.pyplot as plt
import geopandas as gpd

# Setup Flask and ngrok
os.environ["FLASK_DEBUG"] = "true"

app = Flask(__name__, template_folder = '/content/drive/MyDrive/templates')
port = 5000

# Open a ngrok tunnel to the HTTP server
ngrok_authtoken = '2EYf3qVk9mi739HjPwSNZXWAtfy_4jF9NAhGqVVVJmm4YehPW'
ngrok.set_auth_token(ngrok_authtoken)
public_url = ngrok.connect(port).public_url
print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Setup a connection to the database
conn_string = 'mysql+pymysql://{user}:{password}@{host}/{db}?charset={encoding}'.format(
    host = 'db.ipeirotis.org', 
    user = 'student',
    db = 'citibike_fall2017',
    password = 'dwdstudent2015',
    encoding = 'utf8mb4')
engine = create_engine(conn_string)

# Shapefile dataset from NYC Open Data: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq
!curl 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=GeoJSON' -o nyc-neighborhoods.geojson

# Load the shapefile, to create plots on top of the NYC map
df_nyc = gpd.GeoDataFrame.from_file('nyc-neighborhoods.geojson')

# Create the HTML pages

Remember to put the `search_stations.html` and the `list_stations.html` pages under `templates`.

In [ ]:
# Our website has just two pages.


# The main page shows just a big box with a search interface
# The HTML for the page is at 
# https://github.com/ipeirotis/dealing_with_data/blob/master/11-Flask/search_stations.html
@app.route('/')
def home():
    return render_template('search_stations.html')

# This page shows the list of the stations
# Optionally, we also pass a query to the page with 
# the name of the station and we do an approximate search
#
# The HTML for the page is at 
# https://github.com/ipeirotis/dealing_with_data/blob/master/11-Flask/list_stations.html
@app.route('/list_stations',  methods=['GET'])
def list_stations():
  return render_template("list_stations.html")    

# Creating the API calls

Below we write the code for the API calls that will power our webpages.

In [ ]:
# This is a function that connects to the database and returns back a 
# list of Citibike stations. It has an optional paramter "search_name"
# which will limit the number of stations that are returned.
#
# We also have the start/limit parameters that limit the number of 
# returned stations
def get_citibike_stations(search_name=None, start=0, limit=1000):
  sql = "SELECT DISTINCT id, name, capacity, lat, lon  FROM status_fall2017"

  # If we have a search query, we add a condition in SQL
  if search_name:
    sql += " WHERE name LIKE %(station_name_query)s"

  # This (optional) part limits the number of returned stations
  # The offset part allows us to get "later pages" of results
  # if the limit is not sufficient to get everything back
  sql += f" LIMIT {limit} OFFSET {start}"

  with engine.connect() as con:
    # If there is a search query, we populate the parameter in SQL
    # Since we want to have an approximate query uisng LIKE, 
    # we put the wildcard character before and after the search string 
    if search_name: 
      params = {"station_name_query": '%'+search_name+'%' }
      stations = pd.read_sql(sql, con=con, params=params)
    else: 
      stations = pd.read_sql(sql, con=con)

  return stations

In [ ]:
# This is the API call that returns back a list of the Citibike stations
# Optionally we pass a query parameter call "name" which we then use to 
# search for Citibike stations that contain that string in their name

@app.route('/citibike_api')
def citibike_stations():

  search_query = request.args.get('station_name_query')
  
  stations = get_citibike_stations(search_name = search_query)

  # Convert the returned dataframe into a list of dictionaries
  list_of_stations = stations.to_dict(orient='records')
  api_results = {"stations": list_of_stations}

  # We JSON-ify our dictionary and return it as the API response
  return jsonify(api_results)

In [ ]:
# This function gets as input a dataframe with the stations 
# and returns back a plot. The plot in encoded in "base64"
# encoding, so that we can return the image back as part of 
# an API call that returns back JSON data.
def create_station_map_image(stations):

  fig, ax = plt.subplots(figsize=(7, 5))

  # Create the map of NYC neighborhoods
  nyc_map = df_nyc.plot(linewidth=0.1, color='White', edgecolor='Gray', ax = ax)

  # Plot the matching stations
  ax = stations.plot(kind='scatter', x='lon', y='lat', s=3, ax=nyc_map)

  buf = BytesIO()
  fig.savefig(buf, format="png")
  b64encoded = base64.b64encode(buf.getbuffer())
  # Embed the result in the html output.
  image_data = b64encoded.decode("ascii")

  return image_data

In [ ]:
# This API call returns a plot, in JSON format
# Specifically, this returns a map of NYC with the 
# stations that match the query
@app.route('/station_map',  methods=['GET'])
def station_map():

  search_query = request.args.get('station_name_query')
  
  stations = get_citibike_stations(search_name = search_query)

  image_data = create_station_map_image(stations)

  # Create the response. We will put the retrieved data as a list of
  # dictionaries, under the key "stations".
  results = {"image_data": "data:image/png;base64," + image_data}

  # We JSON-ify our dictionary and return it as the API response
  return jsonify(results)

In [ ]:
# Start the webserver.

print(f" * ngrok tunnel '{public_url}' -> 'http://127.0.0.1:{port}'")
app.run(use_reloader=False, port=port)